In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gzip
import s3fs
from tqdm import tqdm

In [ ]:
bucketname = 'borza-hotelcom-data'

In [ ]:
def get_term_list_from_sitemap():
    r = requests.get('https://www.hotels.com/siteindex_www_hotels_com.xml')
    return [re.findall('.com/(.*?)_www',x.text)[0] for 
                x in BeautifulSoup(r.content,'lxml').find_all('loc')]

def get_term_df_from_sitemap(term):
    r = requests.get('https://www.hotels.com/siteindex_www_hotels_com.xml')
    todf = []
    links = [x.text for x in BeautifulSoup(r.content,'lxml').find_all('loc') if '/%s_www' % term in x.text]
    if len(links) > 2:
        links = tqdm(links)
    for link in links:
        r2 = requests.get(link)
        soup = BeautifulSoup(gzip.decompress(r2.content),'lxml')
        todf.append(pd.DataFrame([{'name':x.find('loc').text.split('/')[-2],
                                   'id':x.find('loc').text.split('/')[-3][2:],
                                   'url':x.find('loc').text
                                  } for x in soup.find_all('url')])
        )
    return pd.concat(todf)

def get_full_sitemap():
    full_sitemap = pd.DataFrame()
    for term in tqdm(set(get_term_list_from_sitemap())):
        _df = get_term_df_from_sitemap(term)
        _df['term'] = term
        full_sitemap = full_sitemap.append(_df)
    full_sitemap['categ'] = full_sitemap['url'].apply(lambda x: re.findall(r'.com/(.*?)[0-9|\.|/]',x)[0])
    return full_sitemap

In [ ]:
full_sitemap = get_full_sitemap()

In [ ]:
full_sitemap.to_csv('s3://%s/full_sitemap.csv' % bucketname,index=None)